## Typo-Corrector

After I bought new MacBook, I lost my dominate over the new keyboard and I have a lot of typo during typing. What is interesting in this scenario is that always the mis-characters are in the neighborhood. So I thought a simple Autoencoder should learn this behavior easily.

Then I decided to implement this Jupiter file to see the result. Also, it could be a good exercise :).
So let's do it.

In [1]:
import numpy as np
from keras.models import Model
from keras.layers import Input, LSTM, Dense

Using TensorFlow backend.


First, we need to have all possible character which can by typed mistakenly by neighbor character. Here I defined a dictionary which the key is the character, and values are possible mistake characters.

In [2]:
char_dict ={'q':"wsa",
            'w':"qase",
            'e':"wsdr",
            'r':"edft",
            't':"rfgy",
            'y':"tghu",
            'u':"yhji",
            'i':"ujko",
            'o':"iklp",
            'p':"ol",
            'a':"qsz",
            's':"adeqwxz",
            'd':"esxcfr",
            'f':"rdcvgt",
            'g':"tfvbhy",
            'h':"ygbnju",
            'j':"uhnmki",
            'k':"jmloi",
            'l':"kmp",
            'z':"asx",
            'x':"zsdc",
            'c':"xdfv",
            'v':"cfgb",
            'b':"vghn",
            'n':"bhjm",
            'm':"njkl"}

Then we need to have some words to examine out the idea, the best source is a pre-trained word embedding. I used GloVe. Note that we only need words, not the weights. So I used 50d GloVe word to vector.

In [3]:
vocab = []
with open('glove.6B.50d.txt', mode='r') as file:
    for line in file:
        values = line.strip().split(' ')
        word = values[0].lower()
        if len(word) < 3:
            continue
        vocab.append(word)
        if len(vocab) == 10000:
            break
            

Next step is to produce the raw data. To do that, I defined a function which gets a word as an input and then loops over its characters and replaces this character by mistaken characters.

In [4]:
def mismaker(word):
    mis_words = []
    for i, w in enumerate(list(word)):
        index = char_dict.get(w)
        if index is not None:
            mischars = char_dict[w]

            for m in mischars:
                mis_word = word[:i] + m + word[i + 1:]
                mis_words.append(mis_word)
    
    return mis_words

In [5]:
data = []
for word in vocab:
    row = [word]
    mis_words = mismaker(word)
    row.extend(mis_words)
    data.append(row)

# save data on the disk
with open('data.txt', mode='w') as file:
    for row in data:
        line = ' '.join(row)
        file.write(line+'\n')
del(data)

Create two empty lists for the inputs and the outputs. The inputs are typo words, and outputs are correct words. In autoencoders, we need to provide start and end of sequences. Here, Sequences are a series of characters. The encoder part needs to know where the sequences are ended and decoder needs the start of sequences and end of sequences. I demonstrate it in below picture.

<img src="images/typo.png">

In [6]:
inputs_data = []
outputs_data = []
with open('data.txt', mode='r') as file:
    for line in file:
        words = line.strip().split(' ')
        correct_word = words[0]
        for i in range(1,len(words)):
            inputs_data.append(words[i])
            outputs_data.append('\t'+correct_word+'\n')
            

assert len(inputs_data)==len(outputs_data)
print("There are ", len(inputs_data), "typo words")

There are  286591 typo words


In [7]:
# shuffle data
from random import shuffle
zipped = list(zip(inputs_data,outputs_data))
shuffle(zipped)
inputs_data, outputs_data = zip(*zipped)

In [8]:
# extract unique characters
chars = set()
for word in outputs_data:
    for c in word:
        if c not in chars:
            chars.add(c)

chars = sorted(chars)

# create a character to index dictionary
char2idx = dict()
for c in chars:
    char2idx[c] = len(char2idx)

# create a index to character dictionary
idx2char = {v:k for k,v in char2idx.items()}
print(chars)

['\t', '\n', '$', '&', "'", '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ';', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'é', '’']


### Keras seq2seq
To make life easier, I used [keras sequential model](https://blog.keras.io/a-ten-minute-introduction-to-sequence-to-sequence-learning-in-keras.html). If you don't have info, I highly recommend you to read it.

In [9]:
num_encoder_tokens = num_decoder_tokens = len(char2idx)
max_encoder_seq_length = max([len(word) for word in inputs_data])
max_decoder_seq_length = max([len(word) for word in outputs_data])

In [10]:
encoder_input_data = np.zeros(
    (len(inputs_data), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(inputs_data), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(inputs_data), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

In [11]:
for i, (input_word, output_word) in enumerate(zip(inputs_data, outputs_data)):
    for t, char in enumerate(input_word):
        encoder_input_data[i, t, char2idx[char]] = 1.
    for t, char in enumerate(output_word):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, char2idx[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, char2idx[char]] = 1.

### Hyperparameters
I used only first 5000 words from GloVe word embedding. If we have a larger data, e.g 100K words, we should use a larger LSTM unit. I choose this setting, because of training time.

In [12]:
# hyperparameters
latent_dim = 128
batch_size = 512  
epochs = 50

In [13]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [14]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

In [15]:
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)

Train on 229272 samples, validate on 57319 samples
Epoch 1/50
229272/229272 [==============================] - 20s 88us/step - loss: 1.0376 - val_loss: 0.8413
Epoch 2/50
229272/229272 [==============================] - 17s 75us/step - loss: 0.6844 - val_loss: 0.5513
Epoch 3/50
229272/229272 [==============================] - 17s 75us/step - loss: 0.4756 - val_loss: 0.4041
Epoch 4/50
229272/229272 [==============================] - 17s 75us/step - loss: 0.3529 - val_loss: 0.3251
Epoch 5/50
229272/229272 [==============================] - 17s 75us/step - loss: 0.2770 - val_loss: 0.2367
Epoch 6/50
229272/229272 [==============================] - 17s 75us/step - loss: 0.2255 - val_loss: 0.1945
Epoch 7/50
229272/229272 [==============================] - 17s 75us/step - loss: 0.1898 - val_loss: 0.1663
Epoch 8/50
229272/229272 [==============================] - 17s 75us/step - loss: 0.1636 - val_loss: 0.1436
Epoch 9/50
229272/229272 [==============================] - 17s 75us/step - loss: 0.1

**Caution** I ran this model on GPU 1070. In cpu, It will take more than 5 min for each epoch.

In [16]:
# Save model
model.save('typo-corrector.h5')

/home/alireza/tf14/lib/python3.5/site-packages/keras/engine/topology.py:2368: UserWarning: Layer lstm_2 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_1/while/Exit_2:0' shape=(?, 128) dtype=float32>, <tf.Tensor 'lstm_1/while/Exit_3:0' shape=(?, 128) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  str(node.arguments) + '. They will not be included '


### Evaluation
For evaluation, we need to run a separate graph. Because in this step, we don't have ground truth word and decoder should guess next character, based on previous guessed character (dashed line in the picture above). It will continue until we reach the maximum time step or decoder guess a `\n` character. 

It is good to point it out which decoder in training phase will use ground truth character at each time step.

In [17]:
# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)


In [18]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, char2idx['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = idx2char[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [29]:
def get_one_hot(word):
    oh_word = np.zeros((max_encoder_seq_length, num_encoder_tokens),dtype=np.float32)
    for i, c in enumerate(word):
        oh_word[i,char2idx[c]] = 1.
    return oh_word

In [20]:
for seq_index in range(100):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Typo word:', inputs_data[seq_index])
    print('True word:', outputs_data[seq_index])
    print('Decoded word:', decoded_sentence)

-
Typo word: capabilitoes
True word: 	capabilities

Decoded word: capabilities

-
Typo word: promptong
True word: 	prompting

Decoded word: prompting

-
Typo word: srated
True word: 	stated

Decoded word: stated

-
Typo word: saited
True word: 	waited

Decoded word: waited

-
Typo word: qteps
True word: 	steps

Decoded word: steps

-
Typo word: glags
True word: 	flags

Decoded word: flags

-
Typo word: xpeculated
True word: 	speculated

Decoded word: speculated

-
Typo word: strstegist
True word: 	strategist

Decoded word: streatents

-
Typo word: legislayive
True word: 	legislative

Decoded word: legislative

-
Typo word: titlss
True word: 	titles

Decoded word: titles

-
Typo word: hamfway
True word: 	halfway

Decoded word: hamfway

-
Typo word: albaniana
True word: 	albanians

Decoded word: albanians

-
Typo word: wiss
True word: 	wise

Decoded word: wise

-
Typo word: sjzed
True word: 	sized

Decoded word: sized

-
Typo word: claqses
True word: 	classes

Decoded word: classes

-
Ty

In [33]:
my_typo_sentence = "thid senrence cintains manu typi"
my_words = my_typo_sentence.split()
oh_words = []
for w in my_words:
    oh = get_one_hot(w)
    oh = np.reshape(oh, (1, max_encoder_seq_length, num_encoder_tokens))
    decoded_sentence = decode_sequence(oh)
    print('-')
    print('Typo word:', w)
    print('Decoded word:', decoded_sentence)

-
Typo word: thid
Decoded word: this

-
Typo word: senrence
Decoded word: sentence

-
Typo word: cintains
Decoded word: contains

-
Typo word: manu
Decoded word: many

-
Typo word: typi
Decoded word: typi



It was interesting, isn't it? :)